In [10]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Path to your dataset folder in Google Drive
path = '/content/drive/MyDrive/FakeNews_System/'

# Load datasets
df_fake = pd.read_csv(path + 'Fake.csv')
df_real = pd.read_csv(path + 'True.csv')

# Add labels (1 = fake, 0 = real)
df_fake['label'] = 1
df_real['label'] = 0

# Combine and shuffle
df = pd.concat([df_fake, df_real], ignore_index=True)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Preview
print(df.head())
print("\nDataset shape:", df.shape)
print("\nLabel counts:\n", df['label'].value_counts())


Mounted at /content/drive
                                               title  \
0  Ben Stein Calls Out 9th Circuit Court: Committ...   
1  Trump drops Steve Bannon from National Securit...   
2  Puerto Rico expects U.S. to lift Jones Act shi...   
3   OOPS: Trump Just Accidentally Confirmed He Le...   
4  Donald Trump heads for Scotland to reopen a go...   

                                                text       subject  \
0  21st Century Wire says Ben Stein, reputable pr...       US_News   
1  WASHINGTON (Reuters) - U.S. President Donald T...  politicsNews   
2  (Reuters) - Puerto Rico Governor Ricardo Rosse...  politicsNews   
3  On Monday, Donald Trump once again embarrassed...          News   
4  GLASGOW, Scotland (Reuters) - Most U.S. presid...  politicsNews   

                  date  label  
0    February 13, 2017      1  
1       April 5, 2017       0  
2  September 27, 2017       0  
3         May 22, 2017      1  
4       June 24, 2016       0  

Dataset shape: (44898, 

In [11]:
import nltk
import re
import string
nltk.download('stopwords')
from nltk.corpus import stopwords

# Combine title and text into one column for processing
df['content'] = df['title'] + " " + df['text']

# Remove null values
df.dropna(subset=['content'], inplace=True)

# Get English stopwords
stop_words = set(stopwords.words('english'))

# Function to clean text (without lemmatization)
def clean_text(text):
    text = text.lower()  # lowercase
    text = re.sub(r'\d+', '', text)  # remove numbers
    text = "".join([char for char in text if char not in string.punctuation])  # remove punctuation
    text = re.sub(r'[^\x00-\x7F]+', '', text)  # remove non-ASCII chars
    words = text.split()
    words = [word for word in words if word not in stop_words and len(word) > 2]  # remove stopwords & short words
    text = " ".join(words)
    text = re.sub(r'\s+', ' ', text).strip()  # normalize spaces
    return text

# Apply cleaning
df['content_clean'] = df['content'].apply(clean_text)

# Preview cleaned data
print(df[['content', 'content_clean']].head())
print("\nShape after cleaning:", df.shape)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


                                             content  \
0  Ben Stein Calls Out 9th Circuit Court: Committ...   
1  Trump drops Steve Bannon from National Securit...   
2  Puerto Rico expects U.S. to lift Jones Act shi...   
3   OOPS: Trump Just Accidentally Confirmed He Le...   
4  Donald Trump heads for Scotland to reopen a go...   

                                       content_clean  
0  ben stein calls circuit court committed coup d...  
1  trump drops steve bannon national security cou...  
2  puerto rico expects lift jones act shipping re...  
3  oops trump accidentally confirmed leaked israe...  
4  donald trump heads scotland reopen golf resort...  

Shape after cleaning: (44898, 7)


In [12]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

# Function to lemmatize text
def lemmatize_text(text):
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words]
    return " ".join(words)

# Apply lemmatization on the cleaned content
df['content_lemmatized'] = df['content_clean'].apply(lemmatize_text)

# Preview
print(df[['content_clean', 'content_lemmatized']].head())


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


                                       content_clean  \
0  ben stein calls circuit court committed coup d...   
1  trump drops steve bannon national security cou...   
2  puerto rico expects lift jones act shipping re...   
3  oops trump accidentally confirmed leaked israe...   
4  donald trump heads scotland reopen golf resort...   

                                  content_lemmatized  
0  ben stein call circuit court committed coup dt...  
1  trump drop steve bannon national security coun...  
2  puerto rico expects lift jones act shipping re...  
3  oops trump accidentally confirmed leaked israe...  
4  donald trump head scotland reopen golf resort ...  


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Features: lemmatized text
X = df['content_lemmatized']

# Initialize TF-IDF Vectorizer
# max_features limits the vocabulary size for efficiency
# ngram_range=(1,2) considers unigrams and bigrams
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1,2))

# Fit on the text and transform
X_tfidf = vectorizer.fit_transform(X)

# Preview shape
print("TF-IDF feature matrix shape:", X_tfidf.shape)

# Check feature names
print("Sample features:", vectorizer.get_feature_names_out()[:20])


TF-IDF feature matrix shape: (44898, 10000)
Sample features: ['aaron' 'abadi' 'abandon' 'abandoned' 'abandoning' 'abbas' 'abbott' 'abc'
 'abc news' 'abc week' 'abdel' 'abdullah' 'abe' 'abedin' 'abide' 'ability'
 'able' 'able get' 'aboard' 'abortion']


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Labels
y = df['label']

# Train-test split (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, y, test_size=0.2, random_state=42, stratify=y
)

# Train Logistic Regression
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)

# Predict on test set
y_pred_lr = lr_model.predict(X_test)


In [16]:
from sklearn.svm import LinearSVC

# Train LinearSVC (faster for high-dimensional text)
svm_model = LinearSVC(max_iter=5000, random_state=42)
svm_model.fit(X_train, y_train)

# Predict on test set
y_pred_svm = svm_model.predict(X_test)


In [17]:
from sklearn.metrics import accuracy_score, f1_score, classification_report

print("=== Logistic Regression Results ===")
print("Accuracy:", accuracy_score(y_test, y_pred_lr))
print("F1 Score:", f1_score(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr))

print("\n=== SVM Results ===")
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print("F1 Score:", f1_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))


=== Logistic Regression Results ===
Accuracy: 0.9899777282850779
F1 Score: 0.9903866695150609
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4284
           1       0.99      0.99      0.99      4696

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980


=== SVM Results ===
Accuracy: 0.9963251670378619
F1 Score: 0.9964844998402045
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4284
           1       1.00      1.00      1.00      4696

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980



In [25]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Logistic Regression
precision_lr = precision_score(y_test, y_pred_lr)
recall_lr = recall_score(y_test, y_pred_lr)
f1_lr = f1_score(y_test, y_pred_lr)

# SVM (LinearSVC)
precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)

# Print results
print("=== Logistic Regression ===")
print(f"Precision: {precision_lr:.4f}, Recall: {recall_lr:.4f}, F1: {f1_lr:.4f}")

print("\n=== SVM (LinearSVC) ===")
print(f"Precision: {precision_svm:.4f}, Recall: {recall_svm:.4f}, F1: {f1_svm:.4f}")


=== Logistic Regression ===
Precision: 0.9936, Recall: 0.9872, F1: 0.9904

=== SVM (LinearSVC) ===
Precision: 0.9970, Recall: 0.9960, F1: 0.9965


In [26]:
from sklearn.metrics import accuracy_score, f1_score

# Logistic Regression
accuracy_lr = accuracy_score(y_test, y_pred_lr)
f1_lr = f1_score(y_test, y_pred_lr)

# SVM
accuracy_svm = accuracy_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)

# Print overall results
print("=== Overall Model Performance ===")
print(f"Logistic Regression -> Accuracy: {accuracy_lr:.4f}, F1 Score: {f1_lr:.4f}")
print(f"SVM (LinearSVC)    -> Accuracy: {accuracy_svm:.4f}, F1 Score: {f1_svm:.4f}")


=== Overall Model Performance ===
Logistic Regression -> Accuracy: 0.9900, F1 Score: 0.9904
SVM (LinearSVC)    -> Accuracy: 0.9963, F1 Score: 0.9965


In [27]:
import pickle

# Folder in your Google Drive to save models
save_path = '/content/drive/MyDrive/FakeNews_System/'

# --- Save Logistic Regression model ---
with open(save_path + 'lr_model.pkl', 'wb') as f:
    pickle.dump(lr_model, f)

# --- Save SVM model ---
with open(save_path + 'svm_model.pkl', 'wb') as f:
    pickle.dump(svm_model, f)

# --- Save TF-IDF vectorizer ---
with open(save_path + 'vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

print("Models and vectorizer saved successfully in Drive!")


Models and vectorizer saved successfully in Drive!
